# Clean the original dataset and prepare it for further analyses

In [1]:
import numpy as np
import pandas as pd
import re
import utils
import swifter # swifter makes pandas.DataFrame.apply() slightly faster and communicates the progress

In [2]:
df_orig = pd.read_csv("xandr_segments.csv", sep=",")  # load data
# rename columns
df_orig = df_orig.rename(columns={"Segment Name": "name",
                                  "Data Provider Name": "provider_name",
                                  "Data Provider ID": "provider_id",
                                  "Segment ID": "id"})
df_orig["name_list"] = np.nan  # empty column for itemized segment name

In [3]:
# Providers by frequency
df_orig["provider_name"].value_counts()[:20]

provider_name
Audiences by Oracle (BlueKai, Datalogix, AddThis)    132645
LiveRamp Data Store                                   82363
Grapeshot                                             73569
Nielsen Marketing Cloud                               65610
Eyeota                                                53526
Factual Inc                                           29208
Oracle Customs (1st, BlueKai, Datalogix, AddThis)     26288
Adsquare (Data Provider)                              15246
Dstillery                                             12630
Skydeo, Inc.                                          11972
Semcasting (Data Provider)                            10988
OwnerIQ Inc.                                          10027
Lotame                                                 9523
Peer39                                                 7961
Retargetly                                             7233
AlikeAudience, Inc.                                    7075
KBM Group                 

In [4]:
# Clean segement names (diactrics, stripping, capitalization)
df_orig["name"] = df_orig.swifter.apply(lambda row: utils.clean_segment_name(row["name"]), axis = 1)

Pandas Apply:   0%|          | 0/651463 [00:00<?, ?it/s]

In [5]:
# Segments by a few brokers are multilingual which messes up all kinds of upcoming analyses. All of them follow the same scheme
# and are simply chopped off at this stage.
pruning_re = re.compile(r"\(en\).+\(es\).+\(pt\)")
def prune_segname(row):
    name = row["name"]
    if match := re.search(pruning_re, name):
         row["name"] = name[:match.start(0)].strip()
    return row

df_orig = df_orig.swifter.apply(prune_segname, axis=1)


Pandas Apply:   0%|          | 0/651463 [00:00<?, ?it/s]

In [6]:
# itemize all segment names
df_orig["name_list"] = df_orig.swifter.apply(lambda row: utils.itemize_segment_name(row["name"]), axis = 1)
df_orig = df_orig.set_index("id")

Pandas Apply:   0%|          | 0/651463 [00:00<?, ?it/s]

In [7]:
# remove segments without useful content
useless_segment_indices = df_orig.swifter.apply(lambda x: bool(re.search(utils.useless_segments_re, x["name"])), axis = 1)

df_useless_segments = df_orig[useless_segment_indices].reset_index(drop=True)
df_useless_segments.to_csv("xandr_segments_useless.csv")

df_filtered = df_orig[useless_segment_indices.__invert__()]
df_filtered.to_csv('xandr_segments_itemized.csv')

Pandas Apply:   0%|          | 0/651463 [00:00<?, ?it/s]

In [8]:
# sanity check for segment names that were not itemized
df_failed_itemization = df_filtered[df_filtered.swifter.apply(lambda x: len(x["name_list"]) < 2, axis = 1)]
df_failed_itemization = df_failed_itemization.reset_index(drop=True)
df_failed_itemization.to_csv('xandr_segments_failed_itemization.csv')

Pandas Apply:   0%|          | 0/646024 [00:00<?, ?it/s]